# Loading data

In [92]:
from sparksession import spark

df = spark.read.csv('../data/dataset/DataCoSupplyChainDataset.csv',header=True, inferSchema=True)

df.printSchema()


root
 |-- Type: string (nullable = true)
 |-- Days for shipping (real): integer (nullable = true)
 |-- Days for shipment (scheduled): integer (nullable = true)
 |-- Benefit per order: double (nullable = true)
 |-- Sales per customer: double (nullable = true)
 |-- Delivery Status: string (nullable = true)
 |-- Late_delivery_risk: integer (nullable = true)
 |-- Category Id: integer (nullable = true)
 |-- Category Name: string (nullable = true)
 |-- Customer City: string (nullable = true)
 |-- Customer Country: string (nullable = true)
 |-- Customer Email: string (nullable = true)
 |-- Customer Fname: string (nullable = true)
 |-- Customer Id: integer (nullable = true)
 |-- Customer Lname: string (nullable = true)
 |-- Customer Password: string (nullable = true)
 |-- Customer Segment: string (nullable = true)
 |-- Customer State: string (nullable = true)
 |-- Customer Street: string (nullable = true)
 |-- Customer Zipcode: integer (nullable = true)
 |-- Department Id: integer (nullable = 

In [93]:
df.show()

+--------+------------------------+-----------------------------+-----------------+------------------+-----------------+------------------+-----------+--------------+--------------+----------------+--------------+--------------+-----------+--------------+-----------------+----------------+--------------+--------------------+----------------+-------------+---------------+-----------+------------+------------+----------+-------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+------+----------------+----------------------+--------------+--------------------+---------------+-------------+---------------+-------------------+-------------------+--------------------+------------+-------------+--------------+--------------------------+--------------+
|    Type|Days for shipping (real)|Days for shipment (scheduled)|Benefit per order|Sale

In [94]:

from pyspark.sql import functions as f
df_cleaned = df.filter(f.col('Order Status') != 'CANCELED')
df_cleaned.filter(f.col('Order Status') =='CANCELED' ).count()

df_cleaned.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in df.columns]).show()
# Product Description,Order Zipcode have null values
# df_cleaned = df.drop('Order Zipcode', 'Product Description', 'Customer Email', 'Customer Password' )

df_cleaned = df_cleaned.withColumn(
    'Shipping Mode',
    f.when(f.col('Shipping Mode') == 'Same Day', 'First Class')
    .otherwise(f.col('Shipping Mode'))
)



+----+------------------------+-----------------------------+-----------------+------------------+---------------+------------------+-----------+-------------+-------------+----------------+--------------+--------------+-----------+--------------+-----------------+----------------+--------------+---------------+----------------+-------------+---------------+--------+---------+------+----------+-------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+-----+----------------+----------------------+------------+-----------+------------+-------------+---------------+-------------------+-------------------+-------------+------------+-------------+--------------+--------------------------+-------------+
|Type|Days for shipping (real)|Days for shipment (scheduled)|Benefit per order|Sales per customer|Delivery Status|Late_delivery_risk|Ca

In [95]:
df_cleaned.show()



+--------+------------------------+-----------------------------+-----------------+------------------+-----------------+------------------+-----------+--------------+--------------+----------------+--------------+--------------+-----------+--------------+-----------------+----------------+--------------+--------------------+----------------+-------------+---------------+-----------+------------+------------+----------+-------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+------+----------------+----------------------+--------------+--------------------+---------------+-------------+---------------+-------------------+-------------------+--------------------+------------+-------------+--------------+--------------------------+--------------+
|    Type|Days for shipping (real)|Days for shipment (scheduled)|Benefit per order|Sale

In [96]:
columns_redondantes  = [
    # 'Type',
    'Days for shipping (real)',
    'Delivery Status',
    'Customer Fname',
    'Customer Lname',
    'Customer Email',
    'Customer Password',
    'Order Id',
    'Customer Id',
    'Order Item Id',
    'Order Customer Id',
    'Order Item Cardprod Id',
    'Product Card Id',
    'Product Category Id',
    'Department Id',
    'Product Description',
    'Product Image',
    'Product Name'  ,
    # 'Order Item Total',
    'Order Profit Per Order',
    'Benefit per order',
    'Sales per customer',
    'Order Item Profit Ratio',
    'Customer Country',
    'rder Zipcode',
    'Product Status',
    'Customer State',
    'Customer Street',
    'Customer Zipcode',
    'Department Name',
    'Latitude',
    'Longitude',
    'Market',
    'Order City',
    'Order Country',
    'Order Item Discount',
    'Order Item Discount Rate',
    'Order Item Product Price',
    'Order Item Quantity',
    'Sales',
    'Order Status',
    'Product Card Id',
    'Product Price'
    'Product Status',
    'Shipping date (DateOrders)',
    'Shipping Mode',
    'Category Name',
    'Customer City',
    'Order Zipcode',
    'Order State',
    'Product Price',
    
]


df_columns_cleaned =  df_cleaned.drop(*columns_redondantes)

df_columns_cleaned.show()

+--------+-----------------------------+------------------+-----------+----------------+-----------------------+----------------+--------------+
|    Type|Days for shipment (scheduled)|Late_delivery_risk|Category Id|Customer Segment|order date (DateOrders)|Order Item Total|  Order Region|
+--------+-----------------------------+------------------+-----------+----------------+-----------------------+----------------+--------------+
|   DEBIT|                            4|                 0|         73|        Consumer|        1/31/2018 22:56|     314.6400146|Southeast Asia|
|TRANSFER|                            4|                 1|         73|        Consumer|        1/13/2018 12:27|     311.3599854|    South Asia|
|    CASH|                            4|                 0|         73|        Consumer|        1/13/2018 12:06|     309.7200012|    South Asia|
|   DEBIT|                            4|                 0|         73|     Home Office|        1/13/2018 11:45|     304.8099976| 

In [97]:
df_columns_cleaned = df_columns_cleaned.withColumn(
    'order_date',
    f.to_timestamp('order date (DateOrders)', "M/d/yyyy H:mm")
)

df_columns_cleaned = df_columns_cleaned.withColumn('order_month', f.month('order_date'))

df_columns_cleaned = df_columns_cleaned.drop('order date (DateOrders)', 'order_date')

df_columns_cleaned.show()

+--------+-----------------------------+------------------+-----------+----------------+----------------+--------------+-----------+
|    Type|Days for shipment (scheduled)|Late_delivery_risk|Category Id|Customer Segment|Order Item Total|  Order Region|order_month|
+--------+-----------------------------+------------------+-----------+----------------+----------------+--------------+-----------+
|   DEBIT|                            4|                 0|         73|        Consumer|     314.6400146|Southeast Asia|          1|
|TRANSFER|                            4|                 1|         73|        Consumer|     311.3599854|    South Asia|          1|
|    CASH|                            4|                 0|         73|        Consumer|     309.7200012|    South Asia|          1|
|   DEBIT|                            4|                 0|         73|     Home Office|     304.8099976|       Oceania|          1|
| PAYMENT|                            4|                 0|         7

In [98]:
rslt_minor =df_columns_cleaned.filter(f.col('Late_delivery_risk') == 0)
rslt_major = df_columns_cleaned.filter(f.col('Late_delivery_risk') == 1)

print(rslt_minor.count(), rslt_major.count())


77850 98977


In [99]:
major_count = rslt_major.count()
minor_count = rslt_minor.count()

n_repeats = int(major_count / minor_count)

n_remainder = major_count % minor_count

# replicated_df = rslt_minor.withColumn(
#     "replicator",
#     f.explode(f.array([f.lit(i) for i in range(n_repeats)]))
# ).drop("replicator")

# replicated_df.count()

remainder_df = rslt_minor.orderBy(f.rand(seed=42)).limit(n_remainder)

balanced_df = rslt_major.unionAll(remainder_df)

balanced_df.groupBy("Late_delivery_risk").count().show()

+------------------+-----+
|Late_delivery_risk|count|
+------------------+-----+
|                 1|98977|
|                 0|21127|
+------------------+-----+



In [100]:
balanced_df.show()

+--------+-----------------------------+------------------+-----------+----------------+----------------+--------------+-----------+
|    Type|Days for shipment (scheduled)|Late_delivery_risk|Category Id|Customer Segment|Order Item Total|  Order Region|order_month|
+--------+-----------------------------+------------------+-----------+----------------+----------------+--------------+-----------+
|TRANSFER|                            4|                 1|         73|        Consumer|     311.3599854|    South Asia|          1|
|   DEBIT|                            1|                 1|         73|     Home Office|     288.4200134|  Eastern Asia|          1|
|TRANSFER|                            1|                 1|         73|       Corporate|     285.1400146|  Eastern Asia|          1|
|    CASH|                            2|                 1|         73|       Corporate|     278.5899963|  Eastern Asia|          1|
|    CASH|                            1|                 1|         7

In [101]:
balanced_df.dtypes

[('Type', 'string'),
 ('Days for shipment (scheduled)', 'int'),
 ('Late_delivery_risk', 'int'),
 ('Category Id', 'int'),
 ('Customer Segment', 'string'),
 ('Order Item Total', 'double'),
 ('Order Region', 'string'),
 ('order_month', 'int')]

In [102]:
num_cols = []
cat_cols = []

for col, type in balanced_df.dtypes:
    if type in ['int', 'double'] :
        num_cols.append(col)
    else: 
        cat_cols.append(col)
        
        
cat_indexed = [
    'Customer Segment(indexed)',
    'Order Region(indexed)',
    'Type(indexed)'
    ]


cat_encoded = [    
    'Customer Segment(encoded)',
    'Order Region(encoded)',
    'Type(encoded)']

print(num_cols,cat_cols)


['Days for shipment (scheduled)', 'Late_delivery_risk', 'Category Id', 'Order Item Total', 'order_month'] ['Type', 'Customer Segment', 'Order Region']


In [103]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer, OneHotEncoder

train_df, test_df = balanced_df.randomSplit([0.8, 0.2], seed=42)


indexer = StringIndexer(inputCols=cat_cols, outputCols=cat_indexed)

encoder = OneHotEncoder(inputCols=cat_indexed, outputCols=cat_encoded)

assembler = VectorAssembler(inputCols=num_cols + cat_cols, outputCol='features')

random_forest = RandomForestClassifier(labelCol='Late_delivery_risk', featuresCol='features')




In [104]:

df_with_index = balanced_df.rdd.zipWithIndex().toDF(["row", "index"])
df_test_selected = df_with_index.filter("index <= 3").select("row.*")
balanced_df = df_with_index.filter("index >= 3").select("row.*")

df_test_selected.show()

+--------+-----------------------------+------------------+-----------+----------------+----------------+------------+-----------+
|    Type|Days for shipment (scheduled)|Late_delivery_risk|Category Id|Customer Segment|Order Item Total|Order Region|order_month|
+--------+-----------------------------+------------------+-----------+----------------+----------------+------------+-----------+
|TRANSFER|                            4|                 1|         73|        Consumer|     311.3599854|  South Asia|          1|
|   DEBIT|                            1|                 1|         73|     Home Office|     288.4200134|Eastern Asia|          1|
|TRANSFER|                            1|                 1|         73|       Corporate|     285.1400146|Eastern Asia|          1|
|    CASH|                            2|                 1|         73|       Corporate|     278.5899963|Eastern Asia|          1|
+--------+-----------------------------+------------------+-----------+------------

In [105]:


from pyspark.ml import Pipeline

pipline = Pipeline(stages=[indexer, encoder, assembler, random_forest])

train_df, test_df = balanced_df.randomSplit([0.8, 0.2], seed=42)

model = pipline.fit(train_df)
preds = model.transform(test_df)


IllegalArgumentException: Data type string of column Type is not supported.
Data type string of column Customer Segment is not supported.
Data type string of column Order Region is not supported.